In [139]:
import random

cities = {
    'A': {'days': 3, 'cost': 80},
    'B': {'days': 4, 'cost': 85},
    'C': {'days': 5, 'cost': 100},
    'D': {'days': 2, 'cost': 45},
    'E': {'days': 3, 'cost': 75},
    'F': {'days': 1, 'cost': 25},
    'G': {'days': 6, 'cost': 50},
    'H': {'days': 8, 'cost': 205},
    'I': {'days': 1, 'cost': 24},
    'J': {'days': 7, 'cost': 200},
}

def generate_population(population_size, max_cities, max_days):
    population = []
    # ทำการสุ่ม population เริ่มต้น
    for _ in range(population_size):
        # จำนวนเมืองที่จะสุ่ม ณ ตอนนั้น ซึ่งคิดว่าสุ่มตั้งแต่ 2 - max_cities จะคุ้มและเร็วที่สุด
        num_cities = random.randint(2, max_cities)
        curr_cities = []
        total_days = 0 
        count = 0 # Prevent the infinite loop
        while total_days < max_days and len(curr_cities) < num_cities:
            if count > 10:
                break
            # สุ่มเมืองที่จะไป
            city = random.choice(list(cities.keys()))
            if city in [c for c, _ in curr_cities]:
                count += 1
                continue
            days_to_stay = cities[city]['days']
            if total_days + days_to_stay > max_days:
                count += 1
                continue
            curr_cities.append((city, days_to_stay))
            total_days += days_to_stay
        population.append(curr_cities)
    return population

def fitness_function(select_cities):
    # คำนวณค่า fitness โดยคำนวณจาก จำนวนวันที่เข้าเมือง / ราคาทั้งหมด เพราะคิดว่าน่าจะคิดได้ง่าย
    total_days = 0
    total_cost = 0
    max_days = 14
    visited_cities = set()  # Track unique cities visited
    
    for city, days_to_stay in select_cities:
        total_days += days_to_stay
        total_cost += cities[city]['cost']
        visited_cities.add(city)
    
    days_diff = abs(max_days - total_days)
    days_fitness = 1 / (1 + days_diff)
    cost_fitness = 1 / total_cost
    city_fitness = len(visited_cities) / len(cities)  # Normalize by total number of cities
    
    return days_fitness * cost_fitness * city_fitness

def fitness_calculate(population):
    fitness_values = []
    for select_cities in population:
        fitness_values.append(fitness_function(select_cities))
    return fitness_values

# สำหรับเลือก Parent ในแต่ละรอบ
def roulette_wheel_selection(population, fitness_values, num_parents):
    parents = []
    total_fitness = sum(fitness_values)
    
    # Normalize ให้ค่า fitness อยู่ในช่วง 0-1 สำหรับค่าความน่าจะเป็น
    probs = []
    for value in fitness_values:
        probs.append(value / total_fitness)

    for _ in range(num_parents):
        index = random.choices(range(len(population)), weights=probs)[0]
        parents.append(population[index])

    return parents

# Crossover
def crossover(parent1, parent2):
    # สุ่มจุดที่จะทำการ crossover
    crossover_point = random.randint(1, min(len(parent1), len(parent2)) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation
def mutation(child, max_days):
    # สุ่มเมืองที่จะเข้าไปแทนที่เมืองเดิม
    index = random.randint(0, len(child) - 1)
    count = 0 # Prevent the infinite loop
    while True:
        if count > 10:
            break
        new_city = random.choice(list(cities.keys()))
        if new_city in [c for c, _ in child]:
            count += 1
            continue
        days_to_stay = cities[new_city]['days']
        # ถ้าเมืองที่สุ่มมา สามารถเข้าไปได้โดยไม่เกินจำนวนวันที่กำหนด
        total_days = sum([day for _, day in child])
        if total_days - child[index][1] + days_to_stay <= max_days:
            child[index] = (new_city, days_to_stay)
        else:
            break
        
    return child

### Code

In [140]:
population_size = 10
max_cities = 5
max_days = 14
epochs = 30

# Phase 1: Generate the initial population
init_population = generate_population(population_size, max_cities, max_days)
print(init_population)

iteration_population = init_population

# Phase 2: Selection for breeding offsprings (We will loop here until optimal solution)

for _ in range(epochs):
    # Individuals test for fitness
    fitness_values = fitness_calculate(iteration_population)

    # Selection for reproduction

    num_parents = 2  # Number of parents to select for crossover
    selected_parents = roulette_wheel_selection(iteration_population, fitness_values, num_parents)
    print("Selected Parents:")
    for parent in selected_parents:
        print(parent)

    # Reproduction (Crossover & Mutation)
    offspring = []
    parent1 = selected_parents[0]
    parent2 = selected_parents[1]
    child1, child2 = crossover(parent1, parent2)
    offspring.append(mutation(child1, max_days))
    offspring.append(mutation(child2, max_days))
    
    iteration_population = offspring + iteration_population

# Phase 3 : Evaluation of a population
final_fitness_values = fitness_calculate(iteration_population)
print(final_fitness_values)

[[('F', 1), ('E', 3)], [('D', 2), ('A', 3), ('C', 5), ('B', 4)], [('F', 1), ('J', 7), ('D', 2)], [('C', 5), ('D', 2), ('G', 6), ('F', 1)], [('I', 1), ('B', 4), ('F', 1), ('G', 6), ('D', 2)], [('H', 8), ('D', 2), ('B', 4)], [('G', 6), ('D', 2), ('F', 1), ('A', 3)], [('B', 4), ('J', 7), ('A', 3)], [('J', 7), ('C', 5), ('D', 2)], [('A', 3), ('B', 4), ('C', 5), ('I', 1), ('F', 1)]]
Selected Parents:
[('G', 6), ('D', 2), ('F', 1), ('A', 3)]
[('A', 3), ('B', 4), ('C', 5), ('I', 1), ('F', 1)]
Selected Parents:
[('A', 3), ('B', 4), ('C', 5), ('A', 3)]
[('D', 2), ('A', 3), ('C', 5), ('B', 4)]
Selected Parents:
[('C', 5), ('D', 2), ('G', 6), ('F', 1)]
[('A', 3), ('B', 4), ('C', 5), ('I', 1), ('F', 1)]
Selected Parents:
[('A', 3), ('B', 4), ('C', 5), ('I', 1), ('F', 1)]
[('G', 6), ('D', 2), ('F', 1), ('A', 3)]
Selected Parents:
[('H', 8), ('D', 2), ('B', 4)]
[('A', 3), ('B', 4), ('C', 5), ('I', 1), ('F', 1)]
Selected Parents:
[('J', 7), ('C', 5), ('D', 2)]
[('G', 6), ('D', 2), ('F', 1), ('I', 1),

In [141]:
# Final list

max_value = 0
final_index = 0
for index, value in enumerate(final_fitness_values):
    if value > max_value:
        max_value = value
        final_index = index

final_selected = iteration_population[final_index]

total_cost = 0
total_days = 0

print("Final Selected:")

for city, day in final_selected:
    total_cost += cities[city]['cost']
    total_days += day
    print(f"City: {city} - Days: {day}")

print("Total Cost: ", total_cost)
print("Total Days: ", total_days)

Final Selected:
City: I - Days: 1
City: B - Days: 4
City: F - Days: 1
City: G - Days: 6
City: D - Days: 2
Total Cost:  229
Total Days:  14
